In [58]:
import streamlit as st
import time
import os
from datetime import datetime, timedelta
import seaborn as sns; sns.set()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from plotly.offline import iplot
import pandas as pd
#import statistics as st
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.stats.diagnostic import acorr_ljungbox
import scipy.stats as scs
from math import sqrt
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import warnings
warnings.filterwarnings("ignore")
import sys
import glob
from fbprophet import Prophet

In [59]:
import os
os.getcwd()

'c:\\Users\\DNN\\Data_science\\IGI_quality_control'

In [60]:
def configure_plotly_browser_state():
    import IPython
    display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

In [61]:
#path='F:/Dept/IGI/1-SMT/IGIR-SIS/Line E-series/50200-3025161 Drive pin E25 IGIR-IS/'
path='//Vn01w2k16v18/data/Copyroom/Test_software/Data/20020-3001979 pj185 contact spring igir 97185002/'
files = list(filter(os.path.isfile, glob.glob(path + "*")))
print(len(files))
files.sort(key=lambda x: os.path.getctime(x)) # sort by create date, not modify date
file_df=pd.DataFrame(files,columns=['file_name'])
file_df['Created_time']=file_df.file_name.apply(lambda x: time.ctime(os.path.getctime(x)) )
file_df['Last_modifie']=file_df.file_name.apply(lambda x: time.ctime(os.path.getmtime(x)) )
file_df.Created_time=pd.to_datetime(file_df.Created_time)
cutting_time='2016-07'
file_df=file_df[file_df.Created_time >cutting_time]
all_files=file_df.file_name.tolist()
material_name=all_files[-1]# Problem ?
material_name=material_name.split('/')[-1].split('.')[0]
print('number of files: '+str(len(all_files)))

46
number of files: 46


In [62]:
df = pd.read_excel(all_files[-1], index_col = None, header = 0)
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Part name:,NaN,NaN,Item-configuration:,NaN,Drawing no.:,NaN,Control Date:,NaN,ID no.:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Contact spring,NaN,NaN,20020-3001979,NaN,97185002,NaN,2018-09-08 00:00:00,NaN,867975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Supplier:,Sored,NaN,Batch:,19137,Amount:,NaN,20k,pcs,Inspector:,tnguy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Measurement/Inspection Items,NaN,NaN,NaN,NaN,Item X® - Measure if be required,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Pos.\Item,pos.1,pos.2,pos.3,pos.4,pos.7,pos.6,pos.10,pos.8,pos.5,pos.9,pos.12,pos.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Unit,mm,mm,mm,mm,mm,mm,mm,mm,mm,mm,mm,mm,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,USL,1.9,1.9,1.56,1.17,0.12,0.8,89,0.12,0.8,89,95,95,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
df['Unnamed: 9'][2]

nan

In [64]:
#@st.cache(suppress_st_warning=True,allow_output_mutation=True)

sheet_all={}
n=0
count_file=1
for filename in all_files:
  #print(filename)
  #print('count_file: ',str(count_file) + '/' + str(len(all_files)))
  b='Processing_file: '+ str(count_file) + '/' + str(len(all_files))
  #sys.stdout.write('\r'+b)  
  #st.text(b)
  count_file+=1
  try:
    df = pd.read_excel(filename, index_col = None, header = 0)
    #df.to_excel('file'+str(n)+'.xlsx')
  except:
    print('File cannot be read: '+filename)
    st.text('File cannot be read: '+filename)

  # find begin and end col 
  try:
    # find postion row with Pos/items
    a=df[df.columns[0]].str.contains('Pos.')
    item_row=df[a==True].index[0]
    values_col=df.iloc[item_row,:] # specific row that contain dimension name
    #print(values_col)
    values_col.reset_index(drop=True,inplace=True)
    begin_col=1 # bat dau tu cot thu 1 tro di tu vi tri Pos.\Item
    try:
      end_col=values_col[values_col.isnull() == True].index[0]
    except: # mot so truong hop cot cuoi cung khong phai Nan value thi phai lay cot cuoi cua val_col +1
      end_col=values_col[-1:].index[0]+1
    #print('begin_col',begin_col)
    #print('end_col',end_col)
  except:
    continue

  dim_dict={} # add all value, USL, LSL, UCL... in sheet
  n+=1
  if n==23:n=0
  for name in df.columns[begin_col:end_col]:
    new_df=pd.DataFrame()

    # find begin value collect (check if value is numeric or not):
    try:
        value_begin_row=df[df[df.columns[0]].apply(lambda x: isinstance(x, int))].index[0]
    except:
        value_begin_row= item_row+11
    #Problem:  index 0 is out of bounds for axis 0 with size 0
    new_df['Value']=df[name][value_begin_row:] #19: 5 or 10... value after this row
    # Problem
    new_df=new_df.reset_index(drop=True)
    #print(new_df)
    #print(new_df.applymap(np.isreal))
    try:
      non_nummeric_row=new_df[new_df.applymap(np.isreal).values==False].index[0] # App o cuoi dong value 
    except:
      non_nummeric_row=new_df[new_df.isnull().values==True].index[0] # Nan o cuoi dong value
      #continue
    new_df=new_df[:non_nummeric_row]
    new_df['Value']=new_df['Value'].astype('float32')

    #print(df)
    # Check Date: (find row and column contains date)
    row_date=''
    for i in range(7,10):
      try:
        row_date=df[df[df.columns[i]].apply(lambda x: isinstance(x, datetime))].index[0]
        column_date=i
      except:
        continue
    if row_date=='':
        continue
    #print(row_date,column_date)
    new_df['Date']=df[df.columns[column_date]][row_date]
    # Problem: local variable 'column_date' referenced before assignment ? (solved by check row_date)
    #print(new_df)
    #new_df['Date']=new_df['Date'].apply(lambda x: x.strftime("%Y %m %d %H"))
    new_df['Date']=pd.to_datetime(new_df['Date'])
    new_df['Date']=new_df['Date'].apply(lambda x : x+timedelta(hours=n)) # and hour from 1 to 23 to avoid duplicate

    # Add id(lot) vao dataframe:
    try:
        new_df['ID_No']=df['Unnamed: 9'][2]
    except:
        new_df['ID_No']='' 
    # chuyen vi tri date ra phia ngoai cung, value vao phia trong
    cols = new_df.columns.tolist()
    cols = cols[-2:] + cols[:-2]
    new_df=new_df[cols]

    # Add USL, LSL, UCL, LCL, Mean:
    new_df['USL']=df[name][item_row+2] # max
    new_df['LSL']=df[name][item_row+3] # min
    #new_df['Nominal']=df[name][item_row+8] # min
    new_df[new_df.columns[2:]]=new_df[new_df.columns[2:]].astype('float32')
    dim_name=df[name][item_row]
    dim_dict[dim_name]=new_df
  sheet_all[filename]=dim_dict

#st.text('len(sheet_all): '+str(len(sheet_all)))

In [65]:
new_df

,Date,ID_No,Value,USL,LSL
0,2018-09-08,NaN,93.309998,95.0,92.0
1,2018-09-08,NaN,93.690002,95.0,92.0
2,2018-09-08,NaN,92.800003,95.0,92.0
3,2018-09-08,NaN,93.110001,95.0,92.0
4,2018-09-08,NaN,92.559998,95.0,92.0
5,2018-09-08,NaN,92.110001,95.0,92.0
6,2018-09-08,NaN,92.730003,95.0,92.0
7,2018-09-08,NaN,92.430000,95.0,92.0
8,2018-09-08,NaN,92.419998,95.0,92.0
9,2018-09-08,NaN,93.000000,95.0,92.0


In [66]:
#@st.cache(suppress_st_warning=True,allow_output_mutation=True)
def create_base_file_and_concat(sheet_all):
    
    base_file=sheet_all[list(sheet_all.keys())[-1]].copy() # lay file cuoi cung
    first_dim=list(base_file.keys())[0]
    num_sub_sample=len(base_file[first_dim])
    # concat all process in each weeks based on keys value:
    for file_name in list(sheet_all.keys())[:-1]: # ko tính base week nên từ 1: 
      other_file=sheet_all[file_name]
      for dim_name in other_file.keys(): # dict all process
        for dim_name_base in base_file.keys(): # dict all process
            if dim_name_base==dim_name: 
              base_file[dim_name]=\
              pd.concat([base_file[dim_name],other_file[dim_name]])
              base_file[dim_name]=base_file[dim_name].sort_values(by='Date').reset_index(drop=True).dropna() # drop NaN value in row
    # find max len:            
    max_len=len(base_file[list(base_file.keys())[0]])
    for name in base_file.keys():
        if len(base_file[name]) > max_len: # Invalid comparison between dtype=datetime64[ns] and int
            max_len=len(base_file[name])
    # drop non numeric dim:
    print('before drop non numeric dim: ',base_file.keys())
    dim_drop=[]
    for name in base_file.keys():
      print(name,len(base_file[name]))
      if len(base_file[name]) < 0.1*max_len:
        print('drop dim: ',name)
        dim_drop.append(name)
    for dim in dim_drop:
      base_file.pop(dim,None)
      #base_file=base_file.pop(dim,None)
    print('after drop non numeric dim: ',base_file.keys())
    return base_file,num_sub_sample

base_file,num_sub_sample=create_base_file_and_concat(sheet_all)
print('num_sub_sample: '+ str(num_sub_sample))

before drop non numeric dim:  dict_keys(['pos.1', 'pos.2', 'pos.3', 'pos.4', 'pos.7', 'pos.6', 'pos.10', 'pos.8', 'pos.5', 'pos.9', 'pos.12', 'pos.13'])
pos.1 480
pos.2 480
pos.3 480
pos.4 480
pos.7 60
pos.6 480
pos.10 480
pos.8 60
pos.5 480
pos.9 480
pos.12 463
pos.13 480
after drop non numeric dim:  dict_keys(['pos.1', 'pos.2', 'pos.3', 'pos.4', 'pos.7', 'pos.6', 'pos.10', 'pos.8', 'pos.5', 'pos.9', 'pos.12', 'pos.13'])
num_sub_sample: 21


In [67]:
constants={
    2:1.128,3:1.693,4:2.059,5:2.326,6:2.534,7:2.704,8:2.847, 9: 2.970,10: 3.078,
    11: 3.173,12: 3.258,13: 3.336,14: 3.407,15: 3.472,16: 3.532,17:3.588,18:3.640,
    19:3.689,20:3.735,
}

def process_performance(df,num_sub_sample):
  #print('dim: ',name)
  n=num_sub_sample
  num_sample=n*50 # chuyển từ 25 dữ liệu thành 50 dữ liệu tránh trường hợp các value trùng nhau
  
  df_temp=df[-num_sample:].reset_index(drop=True)
  #print(len(df))
  #print(len(df_temp))
  usl=df_temp.USL[0]
  lsl=df_temp.LSL[0]
  #m=df_temp.Value.mean()
  m=df_temp.Value[-n:].mean() # only last day 

  #Ppk
  sigma=np.std(df_temp.Value)
  Pp = float(usl - lsl) / (6*sigma)
  Ppu = float(usl - m) / (3*sigma)
  Ppl = float(m - lsl) / (3*sigma)
  Ppk = np.min([Ppu, Ppl])
  

  #UCL, LCL, Mean
  k=3
  df['UCL']=df_temp['Value'].mean() + sigma*k
  df['LCL']=df_temp['Value'].mean() - sigma*k
  df['Mean']=df_temp['Value'].mean()

  #Cpk
  
  temp=df_temp.groupby('Date').agg({'Value':['min','max']})
  temp['Range']=temp['Value','max']-temp['Value','min']
  Range=temp['Range'].mean()

  if n <= 20:
    sigma_within = Range/constants[n] # Problem in this line ?
  else:
    sigma_within = Range/constants[20]

  Cp = float(usl - lsl) / (6*sigma_within) #ProblemL float division by zero
  Cpu = float(usl - m) / (3*sigma_within)
  Cpl = float(m - lsl) / (3*sigma_within)
  Cpk = np.min([Cpu, Cpl])
  #print('Pp:{:.2f} , Ppk: {:.2f}'.format(Pp,Ppk))
  #print('Cp:{:.2f} , Cpk:{:.2f}'.format(Cp,Cpk))
  if np.isnan(usl):
    Cpk=Cpl
    Ppk=Ppl
  elif np.isnan(lsl):
    Cpk=Cpu
    Ppk=Ppu
  else:
    Cpk = np.min([Cpu, Cpl])
    Ppk = np.min([Ppu, Ppl])
  Cp=round(Cp,2)
  Cpk=round(Cpk,2)
  Pp=round(Pp,2)
  Ppk=round(Ppk,2)
  return Cp,Cpk,Pp,Ppk

In [68]:
#@st.cache(allow_output_mutation=True)
def create_process_indicator(base_file): 
    process_indicator_dict={}
    process_indicator_df=pd.DataFrame(columns=['Dim','Cp','Pp','Cpk','Ppk'])
    #process_indicator_df.columns=['Dim','Cp','Cpk','Pb','Ppk']
    i=0
    for dim_name in list(base_file.keys()):
      print(dim_name)
      df=base_file[dim_name]
      #try: # object column cannot be calculated process indicator (OK, Not OK...) How to remove object colum in the beginning ?
      Cp,Cpk,Pp,Ppk=process_performance(df,num_sub_sample) 
      #except: continue
      process_indicator_dict[dim_name]=[Cp,Pp,Cpk,Ppk]
      process_indicator_df.loc[i]=dim_name,Cp, Pp, Cpk, Ppk
      i+=1

    process_indicator_df=process_indicator_df.sort_values(by='Ppk').reset_index(drop=True)

    #conver process indicator dict to list name:
    name_list=[]
    for name in process_indicator_dict.keys():
      dim_name='dim: ' + name + ' Cp: ' + str(process_indicator_dict[name][0]) + ' Pp: '+ \
      str(process_indicator_dict[name][1]) +' Cpk: ' \
      + str(process_indicator_dict[name][2]) +' Ppk: '+ str(process_indicator_dict[name][3]) 
      name_list.append(dim_name)
    return process_indicator_df,name_list

process_indicator_df,name_list=create_process_indicator(base_file)

pos.1
pos.2
pos.3
pos.4
pos.7
pos.6
pos.10
pos.8
pos.5
pos.9
pos.12
pos.13


In [69]:


limit = 1.33  # sigma: 4, Yield: 99.99%   
def hightlight_price(row):
    ret = ["" for _ in row.index]
    if row.Cpk < limit or row.Ppk < limit:
      ret[row.index.get_loc("Dim")] = "background-color: yellow"
    if row.Cpk < limit:    
      ret[row.index.get_loc("Cpk")] = "background-color: yellow"
    if row.Ppk < limit:  
      ret[row.index.get_loc("Ppk")] = "background-color: yellow"
    return ret

def highlight_process_indicator(process_indicator_df): 
    return process_indicator_df.style.apply(hightlight_price, axis=1)
process_indicator_df=highlight_process_indicator(process_indicator_df)
process_indicator_df

,Dim,Cp,Pp,Cpk,Ppk
0,pos.9,0.620000,0.360000,-0.620000,-0.360000
1,pos.10,0.680000,0.350000,-0.480000,-0.250000
2,pos.12,0.750000,0.220000,0.320000,0.100000
3,pos.13,1.040000,0.450000,0.370000,0.160000
4,pos.3,1.080000,0.640000,0.680000,0.400000
5,pos.5,0.820000,0.610000,0.600000,0.440000
6,pos.4,1.100000,0.850000,0.780000,0.600000
7,pos.6,0.800000,0.640000,0.800000,0.640000
8,pos.1,0.900000,0.740000,0.820000,0.670000
9,pos.2,1.020000,0.760000,0.930000,0.690000


In [72]:
base_file['pos.9'].groupby('ID_No').Value.mean() 
# Co mot so ma hang do nhieu lan trong ngay, nen group by Date, not by ID_No


ID_No
27302.0        88.152000
27609.0        87.934502
27823.0        88.494499
28015.0        87.817001
28016.0        88.001549
28209.0        88.262001
28328.0        87.870399
28417.0        88.245003
28625.0        88.310501
28896.0        89.350853
28897.0        89.302498
29055.0        89.301498
29056.0        89.086998
29903.0        89.635002
30483.0        89.754997
30540.0        89.815453
30572.0        89.459999
30616.0        89.241501
31190.0        90.041000
36889.0        89.838501
37454.0        89.507004
37547.0        89.454453
37605.0        90.000000
29757+29758    89.770500
Name: Value, dtype: float32

In [73]:
base_file['pos.9'].groupby('Date').aggregate({'Value': 'mean',
                             'ID_No': 'max'})[:20].reset_index()

,Date,Value,ID_No
0,2017-01-24 01:00:00,88.152000,27302
1,2017-02-27 02:00:00,87.934502,27609
2,2017-03-20 03:00:00,88.494499,27823
3,2017-04-11 04:00:00,87.817001,28015
4,2017-04-11 05:00:00,88.001549,28016
5,2017-04-27 06:00:00,88.262001,28209
6,2017-05-08 07:00:00,87.870399,28328
7,2017-05-22 08:00:00,88.245003,28417
8,2017-06-12 09:00:00,88.310501,28625
9,2017-07-19 12:00:00,89.301498,29055


In [74]:
base_file['pos.9'].dtypes

Date     datetime64[ns]
ID_No            object
Value           float32
USL             float32
LSL             float32
UCL             float64
LCL             float64
Mean            float64
dtype: object

In [75]:
configure_plotly_browser_state()
df_dict=base_file
fig = make_subplots(          # Dim name
    rows=len(df_dict), cols=1,
    #shared_xaxes=True, # share same axis
    vertical_spacing=0.05, # adjust spacing between charts
    #column_widths=[0.8, 0.2],
    #subplot_titles=(list(df_dict.keys()))
    subplot_titles=(name_list)
)
i=1
for name in list(df_dict.keys()): #also group
  #print(name)
  df=df_dict[name]
  df=df.sort_values(by=['Date'])
  non_object_column=list(df.select_dtypes(exclude=['object']).columns)
  #print(non_object_column)
  for a in df.columns[1:]:
    if a in non_object_column :
      df[a] = df[a].round(decimals=3)
  df=df.reset_index(drop=True)
  df_group=df.groupby('Date').aggregate({'ID_No': 'max','Value': 'mean','USL': 'mean','LSL': 'mean','UCL': 'mean','LCL': 'mean','Mean': 'mean'})
  #print(df_group)
  #df=df.set_index('Date')
  #if start_date != '' and end_date != '':
  #  df=df[start_date:end_date] 
  #Control chart 1 
  fig.append_trace(go.Scatter(
                          x=df_group.index, y=df_group['Value'],mode='lines+markers',text=df_group['ID_No'],
                          line=dict( color='#4280F5')
                          ),row=i, col=1)
  #USL, LSL
  fig.append_trace(go.Scatter(x=df_group.index, y=df_group['USL'],name='USL ', line=dict( color='#FF5733'),mode='lines'),row=i, col=1)
  fig.append_trace(go.Scatter(x=df_group.index, y=df_group['LSL'],name='LSL ',line=dict( color='#FF5733'),mode='lines'),row=i, col=1)
  #fig.append_trace(go.Scatter(x=df['Datef'], y=df['Nominal'],name='Nominal '+name,line=dict( color='#FF5733')),row=i, col=1)
  # UCL, LCL
  fig.append_trace(go.Scatter(x=df_group.index, y=df_group['UCL'],name='UCL ', line=dict( color='#33C2FF'),mode='lines'),row=i, col=1)
  fig.append_trace(go.Scatter(x=df_group.index, y=df_group['LCL'],name='LCL ', line=dict( color='#33C2FF'),mode='lines'),row=i, col=1)
  fig.append_trace(go.Scatter(x=df_group.index, y=df_group['Mean'],name='Mean ', line=dict( color='#33C2FF'),mode='lines'),row=i, col=1)
  i=i+1


fig.update_layout(height=200*len(df_dict), width=1200, title_text=material_name)
#fig update each process (contain a lot of dim inside)
fig.show()
 

In [141]:
df_group

,ID_No,Value,USL,LSL,UCL,LCL,Mean
Date,,,,,,,
2019-03-02 01:00:00,201902156,0.3020,0.32,0.28,0.307,0.298,0.302
2019-03-11 02:00:00,201903111,0.3016,0.32,0.28,0.307,0.298,0.302
2019-03-15 03:00:00,201903119,0.3008,0.32,0.28,0.307,0.298,0.302
2019-03-25 04:00:00,201903132,0.3016,0.32,0.28,0.307,0.298,0.302
2019-05-11 05:00:00,201905114,0.3042,0.32,0.28,0.307,0.298,0.302
2019-05-21 06:00:00,201905127,0.3036,0.32,0.28,0.307,0.298,0.302
2019-05-22 07:00:00,201905132,0.3066,0.32,0.28,0.307,0.298,0.302
2019-07-16 08:00:00,201907117,0.3024,0.32,0.28,0.307,0.298,0.302
2019-07-22 09:00:00,201907136,0.3036,0.32,0.28,0.307,0.298,0.302


In [55]:
predict_future=3 # months


DFdict_final=base_file
DF_predict_final_={} # only contail predict
#DF_predict_all_final={} # contain predict and past value
for name in DFdict_final.keys(): #also group
  print(name)
  df_temp_2=DFdict_final[name] # gia tri mean thong thuong
  df_temp_2=df_temp_2.groupby('Date').mean()
  df_predict=df_temp_2.copy()
  #df_predict=df_predict.resample('1M').ffill() # gia tri mean da resample
  df_predict=df_predict.resample('M').mean().dropna()
  df_fb=df_predict.reset_index()#
  df_fb=df_fb[[df_fb.columns[0],df_fb.columns[2]]] # old : column 0,1. New: column 1, 2
  df_fb.columns=['ds','y']

  # Prophet will by default fit weekly and yearly seasonalities,
  model = Prophet( #yearly_seasonality=True
                ) #instantiate Prophet with only yearly seasonality as our data is monthly 
  model.fit(df_fb) #fit the model with your dataframe

  # predict for five months in the furure and MS - month start is the frequency
  future = model.make_future_dataframe(periods = predict_future, freq = 'M')  
  # now lets make the forecasts
  forecast = model.predict(future)
  new_forecast=forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
  new_forecast['USL']=df_temp_2['USL'][0]
  new_forecast['LSL']=df_temp_2['LSL'][0]
  #new_forecast['Nominal']=df_temp_2['Mean'][0]
  new_forecast['UCL']=df_temp_2['UCL'][0]
  new_forecast['LCL']=df_temp_2['LCL'][0]
  new_forecast['mean']=df_temp_2['Mean'][0]
  DF_predict_final_[name]=new_forecast 



T
Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
U
Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
W
Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
X
Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
Y
Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
Disabling daily seaso

In [56]:
df_predict

,Value,USL,LSL,UCL,LCL,Mean
Date,,,,,,
2016-07-31,0.045000,0.05,0.03,0.055837,0.036173,0.046005
2016-08-31,0.045812,0.05,0.03,0.055837,0.036173,0.046005
2016-09-30,0.046125,0.05,0.03,0.055837,0.036173,0.046005
2016-10-31,0.045600,0.05,0.03,0.055837,0.036173,0.046005
2016-11-30,0.045917,0.05,0.03,0.055837,0.036173,0.046005
2016-12-31,0.045688,0.05,0.03,0.055837,0.036173,0.046005
2017-01-31,0.045750,0.05,0.03,0.055837,0.036173,0.046005
2017-02-28,0.045000,0.05,0.03,0.055837,0.036173,0.046005
2017-03-31,0.044500,0.05,0.03,0.055837,0.036173,0.046005


In [57]:
df_fb

,ds,y
0,2016-07-31,0.05
1,2016-08-31,0.05
2,2016-09-30,0.05
3,2016-10-31,0.05
4,2016-11-30,0.05
5,2016-12-31,0.05
6,2017-01-31,0.05
7,2017-02-28,0.05
8,2017-03-31,0.05
9,2017-04-30,0.05


In [ ]:
#------------------------------------Prophet Plotly-------------------------------#
@st.cache(suppress_st_warning=True,allow_output_mutation=True)
def line_chart_prophet(base_file,DF_predict_final_,name_list):
    DFdict_final=base_file
    DF_predict_final=DF_predict_final_
    i=1
    #Layout
    fig = make_subplots(
        rows=len(DFdict_final), cols=1,
        #shared_xaxes=True,
        #vertical_spacing=0.05,
        #column_widths=[0.8, 0.2],
        subplot_titles=(name_list)
    )

    for name in DFdict_final.keys(): #also group
      df=DFdict_final[name].copy()
      df=df.sort_values(by=['Date'])
      for a in df.columns[1:]:
        df[a] = df[a].round(decimals=3)
      df=df.groupby('Date').mean()
      #Control chart 1 
      fig.append_trace(go.Scatter(
                              x=df.index, y=df['Value'],
                              mode='lines+markers',
                              name='mean ' + name,line=dict( color='#4280F5')
                              ),row=i, col=1
                    )
      #Predict:
      df_predict=DF_predict_final[name].copy()
      fig.append_trace(go.Scatter(
                          x=df_predict['ds'], y=df_predict['yhat'],
                          mode='lines+markers',
                          name='mean predict ' + name,line=dict( color='#060891')
                          ),row=i, col=1
                )

      #USL, LSL
      fig.append_trace(go.Scatter(x=df_predict['ds'], y=df_predict['USL'],name='USL '+name, line=dict( color='#FF5733'),mode='lines'),row=i, col=1)
      fig.append_trace(go.Scatter(x=df_predict['ds'], y=df_predict['LSL'],name='LSL '+name,line=dict( color='#FF5733'),mode='lines'),row=i, col=1)
      #fig.append_trace(go.Scatter(x=df_predict['ds'], y=df_predict['Nominal'],name='Nominal '+name,line=dict( color='#FF5733')),row=i, col=1)
      # UCL, LCL
      #fig.append_trace(go.Scatter(x=df_predict['ds'], y=df_predict['UCL'],name='UCL '+name, line=dict( color='#33C2FF')),row=i, col=1)
      #fig.append_trace(go.Scatter(x=df_predict['ds'], y=df_predict['LCL'],name='LCL '+name, line=dict( color='#33C2FF')),row=i, col=1)
      #fig.append_trace(go.Scatter(x=df_predict['ds'], y=df_predict['mean'],name='mean '+name, line=dict( color='#33C2FF')),row=i, col=1)
      # Fill between:
      fig.append_trace(go.Scatter(x=df_predict['ds'], y=df_predict['yhat_upper'],name='yhat_upper '+name, line=dict( color='#9FA0F8'),fill=None,mode='lines'),row=i, col=1)
      fig.append_trace(go.Scatter(x=df_predict['ds'], y=df_predict['yhat_lower'],name='yhat_lower '+name, line=dict( color='#9FA0F8'), fill='tonexty',mode='lines'),row=i, col=1)


      #Final layout:
      #fig.update_layout(height=400, width=1400, title_text='Line chart'+name)
      i=i+1

    fig.update_layout(height=200*len(DFdict_final), width=1200, title_text=material_name)
    return fig
fig_new_3=line_chart_prophet(base_file,DF_predict_final_,name_list)
st.plotly_chart(fig_new_3)